In [1]:
import os

import numpy as np
import pandas as pd

from parse_input import parse_itc2007_post_enrolment, parse_itc2007_curriculum_based, parse_xml
from penalty_calc import calculate_total_cost


In [2]:
itc2007_track2_path = '..\\Datasets\\post'
itc2007_track3_path = '..\\Datasets\\curriculum'
itc2019_path = '..\\Datasets\\2019'


def get_all_files(path):
    return [os.path.join(dirpath, file) for dirpath, _, filenames in os.walk(path) for file in filenames]


itc2007_track2_files = get_all_files(itc2007_track2_path)
itc2007_track3_files = get_all_files(itc2007_track3_path)
itc2019_files = get_all_files(itc2019_path)


def parse_problem(dataset, instance_path):
    if dataset == 'itc2007_track2':
        return parse_itc2007_post_enrolment(instance_path)
    elif dataset == 'itc2007_track3':
        return parse_itc2007_curriculum_based(instance_path)
    elif dataset == 'itc2019':
        return parse_xml(instance_path)

In [3]:
random_population_path_2007t2 = 'D:\Desktop\ITC2019\output\initial populations_random_20240417-180757\itc2007 track 2'
random_population_path_2007t3 = 'D:\Desktop\ITC2019\output\initial populations_random_20240417-180757\itc2007 track 3'
random_population_path_2019 = 'D:\Desktop\ITC2019\output\initial populations_random_20240417-180757\itc2019'

valid_population_path_2007t2 = 'D:\Desktop\ITC2019\output\initial populations_valid_20240417-180840\itc2007 track 2'
valid_population_path_2007t3 = 'D:\Desktop\ITC2019\output\initial populations_valid_20240417-180840\itc2007 track 3'
valid_population_path_2019 = 'D:\Desktop\ITC2019\output\initial populations_valid_20240417-180840\itc2019'

In [4]:
df_2007t2 = pd.DataFrame(get_all_files(itc2007_track2_path), columns=['problem path'])

df_2007t3 = pd.DataFrame(get_all_files(itc2007_track3_path), columns=['problem path'])

df_2019 = pd.DataFrame(get_all_files(itc2019_path), columns=['problem path'])

In [5]:
df_2007t2['dataset'] = 'itc2007_track2'
df_2007t3['dataset'] = 'itc2007_track3'
df_2019['dataset'] = 'itc2019'

df_2007t2['problem_file_name'] = df_2007t2['problem path'].apply(lambda x: os.path.basename(x).split('.')[0])
df_2007t3['problem_file_name'] = df_2007t3['problem path'].apply(lambda x: os.path.basename(x).split('.')[0])
df_2019['problem_file_name'] = df_2019['problem path'].apply(lambda x: os.path.basename(x).split('.')[0])

df_2007t2['problem'], df_2007t2['problem_stats'] = zip(
    *df_2007t2.apply(lambda x: parse_problem(x['dataset'], x['problem path']), axis=1))

df_2007t3['problem'], df_2007t3['problem_stats'] , df_2007t3['raw_room_ids'], df_2007t3[
    'raw_course_ids_for_classes'] = zip(
    *df_2007t3.apply(lambda x: parse_problem(x['dataset'], x['problem path']), axis=1))

df_2019['problem'], df_2019['problem_stats'] = zip(
    *df_2019.apply(lambda x: parse_problem(x['dataset'], x['problem path']), axis=1))


In [17]:
df_2007t2['random_population_path'] = df_2007t2['problem_file_name'].apply(
    lambda x: os.path.join(random_population_path_2007t2, x + '.npy'))

df_2007t3['random_population_path'] = df_2007t3['problem_file_name'].apply(
    lambda x: os.path.join(random_population_path_2007t3, x + '.npy'))

df_2019['random_population_path'] = df_2019['problem_file_name'].apply(
    lambda x: os.path.join(random_population_path_2019, x + '.npy'))

df_2007t2['valid_population_path'] = df_2007t2['problem_file_name'].apply(
    lambda x: os.path.join(valid_population_path_2007t2, x + '.npy'))

df_2007t3['valid_population_path'] = df_2007t3['problem_file_name'].apply(
    lambda x: os.path.join(valid_population_path_2007t3, x + '.npy'))

df_2019['valid_population_path'] = df_2019['problem_file_name'].apply(
    lambda x: os.path.join(valid_population_path_2019, x + '.npy'))

df_2007t2['valid_population_times_path'] = df_2007t2['problem_file_name'].apply(
    lambda x: os.path.join(valid_population_path_2007t2, x + ' time.txt'))

df_2007t3['valid_population_times_path'] = df_2007t3['problem_file_name'].apply(
    lambda x: os.path.join(valid_population_path_2007t3, x + ' time.txt'))

df_2019['valid_population_times_path'] = df_2019['problem_file_name'].apply(
    lambda x: os.path.join(valid_population_path_2019, x + ' time.txt'))

In [19]:
df_2007t2['random_population'] = df_2007t2['random_population_path'].apply(lambda x: np.load(x, allow_pickle=False))

df_2007t3['random_population'] = df_2007t3['random_population_path'].apply(lambda x: np.load(x, allow_pickle=False))

df_2019['random_population'] = df_2019['random_population_path'].apply(lambda x: np.load(x, allow_pickle=False))

df_2007t2['valid_population'] = df_2007t2['valid_population_path'].apply(
    lambda x: np.load(x, allow_pickle=False) if os.path.exists(x) else None)

df_2007t3['valid_population'] = df_2007t3['valid_population_path'].apply(
    lambda x: np.load(x, allow_pickle=False) if os.path.exists(x) else None)

df_2019['valid_population'] = df_2019['valid_population_path'].apply(
    lambda x: np.load(x, allow_pickle=False) if os.path.exists(x) else None)    

df_2007t2['valid_population_times'] = df_2007t2['valid_population_times_path'].apply(
    lambda x: pd.read_csv(x, header=None, names=['time','success'], dtype={'time': float, 'success': bool}) if os.path.exists(x) else None)

df_2007t3['valid_population_times'] = df_2007t3['valid_population_times_path'].apply(
    lambda x: pd.read_csv(x, header=None, names=['time','success'], dtype={'time': float, 'success': bool}) if os.path.exists(x) else None)

df_2019['valid_population_times'] = df_2019['valid_population_times_path'].apply(
    lambda x: pd.read_csv(x, header=None, names=['time','success'], dtype={'time': float, 'success': bool}) if os.path.exists(x) else None)

In [8]:
df_2007t2['random_costs'] = df_2007t2.apply(
    lambda x: [calculate_total_cost(x['problem'], sol) for sol in x['random_population']], axis=1)

df_2007t3['random_costs'] = df_2007t3.apply(
    lambda x: [calculate_total_cost(x['problem'], sol) for sol in x['random_population']], axis=1)

df_2019['random_costs'] = df_2019.apply(
    lambda x: [calculate_total_cost(x['problem'], sol) for sol in x['random_population']], axis=1)


df_2007t2['valid_costs'] = df_2007t2.apply(
    lambda x: [calculate_total_cost(x['problem'], sol) for sol in x['valid_population']] if os.path.exists(x['valid_population_path']) else None, axis=1)

df_2007t3['valid_costs'] = df_2007t3.apply(
    lambda x: [calculate_total_cost(x['problem'], sol) for sol in x['valid_population']] if os.path.exists(x['valid_population_path']) else None, axis=1)

df_2019['valid_costs'] = df_2019.apply(
    lambda x: [calculate_total_cost(x['problem'], sol) for sol in x['valid_population']] if os.path.exists(x['valid_population_path']) else None, axis=1)

In [15]:
df_2007t2

,problem path,dataset,problem_file_name,problem,problem_stats,random_population_path,valid_population_path,random_population,valid_population,random_costs,valid_costs,valid_population_times
0,..\Datasets\post\comp-2007-2-1.tim,itc2007_track2,comp-2007-2-1,<models.input.problem.Problem object at 0x0000...,"{'daysInTable': 5, 'timeslotsPerDay': 9, 'even...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[6, 2], [6, 18], [0, 21], [1, 6], [0, 1], [0...",None,"[(2894, 2103), (3283, 2319), (2396, 1914)]",None,None
1,..\Datasets\post\comp-2007-2-10.tim,itc2007_track2,comp-2007-2-10,<models.input.problem.Problem object at 0x0000...,"{'daysInTable': 5, 'timeslotsPerDay': 9, 'even...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 2], [0, 5], [0, 8], [2, 13], [1, 11], [1...",None,"[(2877, 2352), (2713, 2286), (2869, 2475)]",None,None
2,..\Datasets\post\comp-2007-2-11.tim,itc2007_track2,comp-2007-2-11,<models.input.problem.Problem object at 0x0000...,"{'daysInTable': 5, 'timeslotsPerDay': 9, 'even...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[6, 3], [0, 5], [0, 27], [0, 22], [1, 20], [...",None,"[(2220, 3278), (2073, 3402), (2695, 4206)]",None,None
3,..\Datasets\post\comp-2007-2-12.tim,itc2007_track2,comp-2007-2-12,<models.input.problem.Problem object at 0x0000...,"{'daysInTable': 5, 'timeslotsPerDay': 9, 'even...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 3], [0, 25], [9, 13], [1, 19], [1, 9], [...",None,"[(2511, 3865), (1877, 3701), (2314, 3622)]",None,None
4,..\Datasets\post\comp-2007-2-13.tim,itc2007_track2,comp-2007-2-13,<models.input.problem.Problem object at 0x0000...,"{'daysInTable': 5, 'timeslotsPerDay': 9, 'even...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 18], [10, 4], [16, 7], [12, 1], [5, 17],...",None,"[(1839, 1267), (1648, 1241), (1733, 1468)]",None,None
5,..\Datasets\post\comp-2007-2-14.tim,itc2007_track2,comp-2007-2-14,<models.input.problem.Problem object at 0x0000...,"{'daysInTable': 5, 'timeslotsPerDay': 9, 'even...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 14], [0, 11], [0, 19], [0, 0], [0, 9], [...",None,"[(1611, 1261), (1699, 1347), (1439, 1152)]",None,None
6,..\Datasets\post\comp-2007-2-15.tim,itc2007_track2,comp-2007-2-15,<models.input.problem.Problem object at 0x0000...,"{'daysInTable': 5, 'timeslotsPerDay': 9, 'even...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 18], [0, 8], [0, 6], [0, 0], [2, 13], [0...",None,"[(1238, 2191), (1393, 2517), (1330, 2485)]",None,None
7,..\Datasets\post\comp-2007-2-16.tim,itc2007_track2,comp-2007-2-16,<models.input.problem.Problem object at 0x0000...,"{'daysInTable': 5, 'timeslotsPerDay': 9, 'even...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 16], [0, 10], [0, 0], [4, 6], [4, 12], [...",None,"[(1250, 2242), (1628, 2472), (1464, 2382)]",None,None
8,..\Datasets\post\comp-2007-2-17.tim,itc2007_track2,comp-2007-2-17,<models.input.problem.Problem object at 0x0000...,"{'daysInTable': 5, 'timeslotsPerDay': 9, 'even...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 4], [1, 7], [7, 23], [1, 13], [0, 11], [...",None,"[(1856, 1974), (1670, 1988), (1506, 1398)]",None,None
9,..\Datasets\post\comp-2007-2-18.tim,itc2007_track2,comp-2007-2-18,<models.input.problem.Problem object at 0x0000...,"{'daysInTable': 5, 'timeslotsPerDay': 9, 'even...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 1], [6, 26], [1, 16], [1, 11], [0, 27], ...",None,"[(2362, 1890), (2362, 2383), (2589, 2280)]",No

In [21]:
df_2007t3

,problem path,dataset,problem_file_name,problem,problem_stats,raw_room_ids,raw_course_ids_for_classes,random_population_path,valid_population_path,random_population,valid_population,random_costs,valid_costs,valid_population_times,valid_population_times_path
0,..\Datasets\curriculum\comp01.ctt,itc2007_track3,comp01,<models.input.problem.Problem object at 0x0000...,"{'name': 'Fis0506-1', 'course_count': 30, 'roo...","[B, C, E, F, G, S]","[c0001, c0001, c0001, c0001, c0001, c0001, c00...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[1, 13], [0, 20], [1, 23], [4, 13], [2, 21],...","[[[0, 20], [3, 22], [2, 15], [2, 18], [4, 21],...","[(179, 2618), (138, 2354), (158, 2294)]","[(0, 2510), (0, 2569), (0, 2223)]",time success 0 0.799999 True 1 0...,D:\Desktop\ITC2019\output\initial populations_...
1,..\Datasets\curriculum\comp02.ctt,itc2007_track3,comp02,<models.input.problem.Problem object at 0x0000...,"{'name': 'Ing0203-2', 'course_count': 82, 'roo...","[36, 37, 38, 31, 27, B, D, E, F, G, A, L, 50, ...","[c0131, c0131, c0131, c0211, c0211, c0211, c00...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[3, 10], [12, 17], [7, 18], [13, 4], [10, 7]...","[[[6, 5], [9, 8], [3, 22], [1, 2], [2, 7], [10...","[(335, 9342), (417, 7733), (364, 8367)]","[(0, 7690), (0, 7958), (0, 7608)]",time success 0 2.646014 True 1 3...,D:\Desktop\ITC2019\output\initial populations_...
2,..\Datasets\curriculum\comp03.ctt,itc2007_track3,comp03,<models.input.problem.Problem object at 0x0000...,"{'name': 'Ing0304-1', 'course_count': 72, 'roo...","[36, 37, 38, 31, 27, B, D, E, F, G, A, L, 50, ...","[ChiGenAn, ChiGenAn, ChiGenAn, GeoAn, GeoAn, G...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 0], [2, 16], [2, 8], [1, 5], [6, 24], [1...","[[[7, 6], [15, 3], [4, 14], [10, 14], [6, 24],...","[(255, 7079), (328, 6412), (379, 6370)]","[(0, 6423), (0, 6235), (0, 6227)]",time success 0 2.102012 True 1 2...,D:\Desktop\ITC2019\output\initial populations_...
3,..\Datasets\curriculum\comp04.ctt,itc2007_track3,comp04,<models.input.problem.Problem object at 0x0000...,"{'name': 'Ing0405-3', 'course_count': 79, 'roo...","[25, 36, 37, 38, 31, 27, 51, B, D, F, G, A, L,...","[c0012, c0012, c0012, c0013, c0013, c0013, c00...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[17, 6], [14, 5], [2, 0], [17, 16], [15, 14]...","[[[0, 18], [14, 5], [11, 4], [3, 7], [10, 13],...","[(260, 5349), (272, 5718), (253, 5737)]","[(0, 5806), (0, 5363), (0, 5060)]",time success 0 2.816985 True 1 2...,D:\Desktop\ITC2019\output\initial populations_...
4,..\Datasets\curriculum\comp05.ctt,itc2007_track3,comp05,<models.input.problem.Problem object at 0x0000...,"{'name': 'Let0405-1', 'course_count': 54, 'roo...","[10, 14, 15, C1, L, M, O, B, A]","[LinTed1, AlfInf, LetIta1, LetIta1, LetIta1, F...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[3, 1], [7, 25], [4, 7], [7, 2], [8, 4], [4,...",None,"[(412, 8904), (366, 9596), (450, 9409)]",None,None,D:\Desktop\ITC2019\output\initial populations_...
5,..\Datasets\curriculum\comp06.ctt,itc2007_track3,comp06,<models.input.problem.Problem object at 0x0000...,"{'name': 'Ing0506-1', 'course_count': 108, 'ro...","[25, 36, 37, 38, 31, 27, 51, B, D, F, G, A, L,...","[c0006, c0006, c0006, c0011, c0011, c0011, c00...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[5, 14], [6, 7], [13, 5], [5, 1], [10, 3], [...","[[[3, 12], [14, 6], [8, 10], [17, 7], [1, 11],...","[(370, 6879), (469, 7107), (390, 7660)]","[(0, 7095), (0, 7081), (0, 6251)]",time success 0 4.460001 True 1 4...,D:\Desktop\ITC2019\output\initial populations_...
6,..\Datasets\curriculum\comp07.ctt,itc2007_track3,comp07,<models.input.problem.Problem object at 0x0000...,

In [20]:
df_2019

,problem path,dataset,problem_file_name,problem,problem_stats,random_population_path,valid_population_path,random_population,valid_population,random_costs,valid_costs,valid_population_times,valid_population_times_path
0,..\Datasets\2019\1.early\agh-fis-spr17.xml,itc2019,agh-fis-spr17,<models.input.problem.Problem object at 0x0000...,"{'name': 'agh-fis-spr17', 'student_count': 164...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[2, 200], [-1, 33], [-1, 62], [-1, 54], [-1,...","[[[4, 167], [-1, 143], [-1, 129], [-1, 154], [...","[(818, 16137), (785, 13700), (770, 15827)]","[(0, 17818), (0, 17921), (0, 18109)]",time success 0 1297.361632 Tru...,D:\Desktop\ITC2019\output\initial populations_...
1,..\Datasets\2019\1.early\agh-ggis-spr17.xml,itc2019,agh-ggis-spr17,<models.input.problem.Problem object at 0x0000...,"{'name': 'agh-ggis-spr17', 'student_count': 21...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 13], [0, 0], [0, 0], [0, 0], [0, 0], [0,...",None,"[(2088, 94571), (2137, 95870), (2061, 98028)]",None,None,D:\Desktop\ITC2019\output\initial populations_...
2,..\Datasets\2019\1.early\bet-fal17.xml,itc2019,bet-fal17,<models.input.problem.Problem object at 0x0000...,"{'name': 'bet-fal17', 'student_count': 3018, '...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[-1, 0], [31, 37], [31, 34], [30, 20], [4, 2...",None,"[(1499, 379609), (1381, 379458), (1439, 391005)]",None,None,D:\Desktop\ITC2019\output\initial populations_...
3,..\Datasets\2019\1.early\iku-fal17.xml,itc2019,iku-fal17,<models.input.problem.Problem object at 0x0000...,"{'name': 'iku-fal17', 'student_count': 0, 'avg...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 61], [0, 13], [3, 42], [1, 5], [5, 33], ...",None,"[(3298, 171821), (3250, 173808), (3097, 171982)]",None,None,D:\Desktop\ITC2019\output\initial populations_...
4,..\Datasets\2019\1.early\mary-spr17.xml,itc2019,mary-spr17,<models.input.problem.Problem object at 0x0000...,"{'name': 'mary-spr17', 'student_count': 3666, ...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[4, 8], [9, 11], [8, 11], [10, 17], [15, 9],...",None,"[(2403, 78531), (1962, 57518), (1477, 65391)]",None,None,D:\Desktop\ITC2019\output\initial populations_...
5,..\Datasets\2019\1.early\muni-fi-spr16.xml,itc2019,muni-fi-spr16,<models.input.problem.Problem object at 0x0000...,"{'name': 'muni-fi-spr16', 'student_count': 154...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 0], [-1, 6], [-1, 0], [-1, 0], [-1, 0], ...","[[[0, 0], [-1, 17], [-1, 0], [-1, 0], [-1, 0],...","[(495, 9392), (441, 9421), (448, 10201)]","[(0, 9774), (0, 9988), (0, 10238)]",time success 0 7.227103 True 1 6...,D:\Desktop\ITC2019\output\initial populations_...
6,..\Datasets\2019\1.early\muni-fsps-spr17.xml,itc2019,muni-fsps-spr17,<models.input.problem.Problem object at 0x0000...,"{'name': 'muni-fsps-spr17', 'student_count': 8...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[0, 14], [0, 8], [0, 0], [1, 0], [2, 0], [-1...","[[[0, 2], [0, 4], [0, 0], [0, 1], [2, 0], [-1,...","[(473, 13703), (496, 12097), (452, 11946)]","[(0, 9721), (0, 10533), (0, 11157)]",time success 0 6.829000 True 1 6...,D:\Desktop\ITC2019\output\initial populations_...
7,..\Datasets\2019\1.early\muni-pdf-spr16c.xml,itc2019,muni-pdf-spr16c,<models.input.problem.Problem object at 0x0000...,"{'name': 'muni-pdf-spr16c', 'student_count': 2...",D:\Desktop\ITC2019\output\initial populations_...,D:\Desktop\ITC2019\output\initial populations_...,"[[[1, 6], [0, 0], [-1, 12], [0, 0], [-1, 8], [...",None,"[(2205, 290315), (2136, 281570), (2380, 290559)]",None,None,D:\Desktop\ITC2019\output\initial populatio